In [2]:
### imports
import warnings
warnings.simplefilter('ignore')
import os
import numpy as np
import pandas as pd
import joblib
import yaml

In [3]:
def make_main_synthetic_results_table(config_path, results_dir, metric_funcs):

    with open(config_path) as f:
        config = yaml.safe_load(f)

    setup_options = config['setup-options']

    for clever in setup_options['clever']:
        resd = {}
        for dgp_version in setup_options['dgp-version']:
            resd[dgp_version] = {}
            for n in setup_options['n']:
                lambda_l2_h = .1/n**(.9)
                # print(lambda_l2_h)
                nkey = f'$n={n}$'
                resd[dgp_version][nkey] = {}
                for iv_strength in setup_options['iv-strength']:
                    setup = {
                        'clever': clever,
                        'dgp-version': dgp_version,
                        'n': n,
                        'iv-strength': iv_strength,
                    }
                    results_fname = '__'.join(f'{k_}_{setup[k_]}' for k_ in setup_options.keys()) + '.jbl'
                    results_path = os.path.join(results_dir, results_fname)
                    true, results = joblib.load(results_path)
                    ivkey = f'$\rho={iv_strength}$'
                    resd[dgp_version][nkey][ivkey] = {}
                    for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
                        # get array of results for given method over experiment replications
                        data = np.array([r[it] for r in results])
                        resd[dgp_version][nkey][ivkey][method] = {
                            metric_: metric_func_(data, true, method)
                            for metric_, metric_func_ in metric_funcs.items()
                        }
                        resd[dgp_version][nkey][ivkey] = pd.DataFrame(resd[dgp_version][nkey][ivkey])
                resd[dgp_version][nkey] = pd.concat(resd[dgp_version][nkey])
            resd[dgp_version] = pd.concat(resd[dgp_version])

        display(pd.concat(resd).unstack(level=3))
        print(pd.concat(resd).unstack(level=3).to_latex(bold_rows=True, multirow=True,
                                                        multicolumn=True, escape=False,
                                                        column_format='lll||lll|lll|lll|lll|',
                                                        multicolumn_format='c|'))


def get_cov(data, true, method):
    if method in ['dr', 'tmle']:
        return f'{100*np.mean((data[:, 1] <= true) & (true <= data[:, 2])):.0f}'
    else:
        return 'NA'


def get_rmse(data, true, method):
    rmse = np.sqrt(np.mean((data[:, 0] - true) ** 2))
    return f'{rmse:.3f}'


def get_bias(data, true, method):
    abs_bias = np.abs(np.mean((data[:, 0] - true)))
    return f'{abs_bias:.3f}'


def get_normalized_rmse(data, true, method):
    rmse = np.sqrt(np.mean((data[:, 0] - true) ** 2))
    norm_rmse = rmse / np.abs(true)
    return f'{norm_rmse:.3f}'


def get_normalized_bias(data, true, method):
    abs_bias = np.abs(np.mean((data[:, 0] - true)))
    norm_abs_bias = abs_bias / np.abs(true)
    return f'{norm_abs_bias:.3f}'
    
    

In [3]:
# results for strong instruments experiment

make_main_synthetic_results_table(
    config_path='experiment-configs/strong-instruments-config.yaml',
    results_dir='results/strong-instruments',
    metric_funcs={
        'cov': get_cov,
        'rmse': get_rmse,
        'bias': get_bias,
    },
)

KeyError: 'fname'

In [4]:
# results for clever instruments experiment

make_results_table(
    config_path='experiment-configs/clever-instruments-config.yaml',
    results_dir='results/clever-instruments',
    metric_funcs={
        'cov': get_cov,
        'rmse': get_rmse,
        'bias': get_bias,
    },
)

dr               tmle               ipw         \
                            cov   rmse   bias  cov   rmse   bias cov   rmse   
abs     $n=500$  $\rho=0.2$  91  0.163  0.010   80  0.536  0.031  NA  0.164   
                 $\rho=0.5$  94  0.071  0.014   92  0.074  0.013  NA  0.063   
        $n=1000$ $\rho=0.2$  90  0.137  0.007   86  0.202  0.035  NA  0.131   
                 $\rho=0.5$  98  0.046  0.009   98  0.047  0.008  NA  0.047   
        $n=2000$ $\rho=0.2$  89  0.096  0.006   86  0.114  0.007  NA  0.098   
                 $\rho=0.5$  93  0.033  0.001   93  0.034  0.000  NA  0.037   
2dpoly  $n=500$  $\rho=0.2$  80  0.219  0.030   75  0.475  0.018  NA  0.219   
                 $\rho=0.5$  88  0.092  0.009   87  0.092  0.010  NA  0.097   
        $n=1000$ $\rho=0.2$  90  0.152  0.013   87  0.206  0.021  NA  0.162   
                 $\rho=0.5$  90  0.056  0.011   89  0.056  0.010  NA  0.069   
        $n=2000$ $\rho=0.2$  89  0.113  0.015   91  0.122  0.004  NA  0.130   
                 $\rho=0.5$  92  0.043  0.000   92  0.044  0.000  NA  0.067   
sigmoid $n=500$  $\rho=0.2$  93  0.136  0.019   83  0.449  0.002  NA  0.158   
                 $\rho=0.5$  96  0.046  0.010   92  0.048  0.009  NA  0.049   
        $n=1000$ $\rho=0.2$  91  0.101  0.008   87  0.152  0.028  NA  0.119   
                 $\rho=0.5$  95  0.036  0.001   95  0.037  0.001  NA  0.035   
        $n=2000$ $\rho=0.2$  85  0.078  0.003   84  0.091  0.013  NA  0.086   
                 $\rho=0.5$  92  0.024  0.000   92  0.024  0.000  NA  0.021   
sin     $n=500$  $\rho=0.2$  90  0.156  0.018   82  0.556  0.003  NA  0.155   
                 $\rho=0.5$  96  0.046  0.002   92  0.047  0.001  NA  0.046   
        $n=1000$ $\rho=0.2$  92  0.108  0.006   87  0.169  0.031  NA  0.118   
                 $\rho=0.5$  92  0.036  0.002   92  0.037  0.002  NA  0.036   
        $n=2000$ $\rho=0.2$  85  0.086  0.005   86  0.103  0.017  NA  0.091   
                 $\rho=0.5$  95  0.024  0.002   92  0.024  0.002  NA  0.022   

                                   direct                
                              bias    cov   rmse   bias  
abs     $n=500$  $\rho=0.2$  0.022     NA  0.110  0.035  
                 $\rho=0.5$  0.016     NA  0.066  0.006  
        $n=1000$ $\rho=0.2$  0.008     NA  0.097  0.028  
                 $\rho=0.5$  0.005     NA  0.039  0.002  
        $n=2000$ $\rho=0.2$  0.001     NA  0.090  0.066  
                 $\rho=0.5$  0.001     NA  0.026  0.006  
2dpoly  $n=500$  $\rho=0.2$  0.132     NA  0.213  0.105  
                 $\rho=0.5$  0.024     NA  0.105  0.006  
        $n=1000$ $\rho=0.2$  0.073     NA  0.136  0.081  
                 $\rho=0.5$  0.012     NA  0.055  0.006  
        $n=2000$ $\rho=0.2$  0.050     NA  0.118  0.078  
                 $\rho=0.5$  0.009     NA  0.035  0.000  
sigmoid $n=500$  $\rho=0.2$  0.068     NA  0.113  0.015  
                 $\rho=0.5$  0.021     NA  0.051  0.012  
        $n=1000$ $\rho=0.2$  0.037     NA  0.072  0.010  
                 $\rho=0.5$  0.007     NA  0.030  0.002  
        $n=2000$ $\rho=0.2$  0.024     NA  0.065  0.032  
                 $\rho=0.5$  0.001     NA  0.021  0.004  
sin     $n=500$  $\rho=0.2$  0.049     NA  0.103  0.009  
                 $\rho=0.5$  0.013     NA  0.049  0.007  
        $n=1000$ $\rho=0.2$  0.026     NA  0.078  0.016  
                 $\rho=0.5$  0.001     NA  0.035  0.002  
        $n=2000$ $\rho=0.2$  0.021     NA  0.069  0.043  
                 $\rho=0.5$  0.001     NA  0.022  0.011

\begin{tabular}{lll||lll|lll|lll|lll|}
\toprule
 &  &  & \multicolumn{3}{c|}{dr} & \multicolumn{3}{c|}{tmle} & \multicolumn{3}{c|}{ipw} & \multicolumn{3}{c|}{direct} \\
 &  &  & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias \\
\midrule
ho=0.2$} & 91 & 0.163 & 0.010 & 80 & 0.536 & 0.031 & NA & 0.164 & 0.022 & NA & 0.110 & 0.035 \\
ho=0.5$} & 94 & 0.071 & 0.014 & 92 & 0.074 & 0.013 & NA & 0.063 & 0.016 & NA & 0.066 & 0.006 \\
\cline{2-15}
ho=0.2$} & 90 & 0.137 & 0.007 & 86 & 0.202 & 0.035 & NA & 0.131 & 0.008 & NA & 0.097 & 0.028 \\
ho=0.5$} & 98 & 0.046 & 0.009 & 98 & 0.047 & 0.008 & NA & 0.047 & 0.005 & NA & 0.039 & 0.002 \\
\cline{2-15}
ho=0.2$} & 89 & 0.096 & 0.006 & 86 & 0.114 & 0.007 & NA & 0.098 & 0.001 & NA & 0.090 & 0.066 \\
ho=0.5$} & 93 & 0.033 & 0.001 & 93 & 0.034 & 0.000 & NA & 0.037 & 0.001 & NA & 0.026 & 0.006 \\
\cline{1-15} \cline{2-15}
ho=0.2$} & 80 & 0.219 & 0.030 & 75 & 0.475 & 0.018 & NA & 0.219 & 0.132 & NA & 0.213 & 0.105 \\
ho=0.5$}

In [5]:
# results for weak instruments experiment

make_results_table(
    config_path='experiment-configs/weak-instruments-config.yaml',
    results_dir='results/weak-instruments',
    metric_funcs={
        'cov': get_cov,
        '$\\frac{\\textbf{rmse}}{|\\theta^*|}$': get_normalized_rmse,
        '$\\frac{\\textbf{bias}}{|\\theta^*|}$': get_normalized_bias,
    },
)

dr                                     \
                             cov $\frac{\textbf{rmse}}{|\theta^*|}$   
2dpoly $n=2000$  $\rho=0.05$  88                              1.868   
                 $\rho=0.1$   89                              0.740   
       $n=20000$ $\rho=0.05$  89                              1.326   
                 $\rho=0.1$   92                              0.343   

                                                                tmle  \
                             $\frac{\textbf{bias}}{|\theta^*|}$  cov   
2dpoly $n=2000$  $\rho=0.05$                              0.450   91   
                 $\rho=0.1$                               0.135   83   
       $n=20000$ $\rho=0.05$                              0.023   96   
                 $\rho=0.1$                               0.010   91   

                                                                 \
                             $\frac{\textbf{rmse}}{|\theta^*|}$   
2dpoly $n=2000$  $\rho=0.05$                             11.321   
                 $\rho=0.1$                              25.165   
       $n=20000$ $\rho=0.05$                              3.341   
                 $\rho=0.1$                               0.374   

                                                                ipw  \
                             $\frac{\textbf{bias}}{|\theta^*|}$ cov   
2dpoly $n=2000$  $\rho=0.05$                              1.845  NA   
                 $\rho=0.1$                               2.259  NA   
       $n=20000$ $\rho=0.05$                              0.082  NA   
                 $\rho=0.1$                               0.011  NA   

                                                                 \
                             $\frac{\textbf{rmse}}{|\theta^*|}$   
2dpoly $n=2000$  $\rho=0.05$                              1.896   
                 $\rho=0.1$                               0.837   
       $n=20000$ $\rho=0.05$                              1.410   
                 $\rho=0.1$                               0.373   

                                                                direct  \
                             $\frac{\textbf{bias}}{|\theta^*|}$    cov   
2dpoly $n=2000$  $\rho=0.05$                              0.660     NA   
                 $\rho=0.1$                               0.325     NA   
       $n=20000$ $\rho=0.05$                              0.041     NA   
                 $\rho=0.1$                               0.041     NA   

                                                                 \
                             $\frac{\textbf{rmse}}{|\theta^*|}$   
2dpoly $n=2000$  $\rho=0.05$                              0.710   
                 $\rho=0.1$                               0.452   
       $n=20000$ $\rho=0.05$                              0.339   
                 $\rho=0.1$                               0.127   

                                                                 
                             $\frac{\textbf{bias}}{|\theta^*|}$  
2dpoly $n=2000$  $\rho=0.05$                              0.567  
                 $\rho=0.1$                               0.328  
       $n=20000$ $\rho=0.05$                              0.229  
                 $\rho=0.1$                               0.038

\begin{tabular}{lll||lll|lll|lll|lll|}
\toprule
 &  &  & \multicolumn{3}{c|}{dr} & \multicolumn{3}{c|}{tmle} & \multicolumn{3}{c|}{ipw} & \multicolumn{3}{c|}{direct} \\
 &  &  & cov & $\frac{\textbf{rmse}}{|\theta^*|}$ & $\frac{\textbf{bias}}{|\theta^*|}$ & cov & $\frac{\textbf{rmse}}{|\theta^*|}$ & $\frac{\textbf{bias}}{|\theta^*|}$ & cov & $\frac{\textbf{rmse}}{|\theta^*|}$ & $\frac{\textbf{bias}}{|\theta^*|}$ & cov & $\frac{\textbf{rmse}}{|\theta^*|}$ & $\frac{\textbf{bias}}{|\theta^*|}$ \\
\midrule
ho=0.05$} & 88 & 1.868 & 0.450 & 91 & 11.321 & 1.845 & NA & 1.896 & 0.660 & NA & 0.710 & 0.567 \\
ho=0.1$} & 89 & 0.740 & 0.135 & 83 & 25.165 & 2.259 & NA & 0.837 & 0.325 & NA & 0.452 & 0.328 \\
\cline{2-15}
ho=0.05$} & 89 & 1.326 & 0.023 & 96 & 3.341 & 0.082 & NA & 1.410 & 0.041 & NA & 0.339 & 0.229 \\
ho=0.1$} & 92 & 0.343 & 0.010 & 91 & 0.374 & 0.011 & NA & 0.373 & 0.041 & NA & 0.127 & 0.038 \\
\cline{1-15} \cline{2-15}
\bottomrule
\end{tabular}



In [ ]:
# results for partially linear IV experiments

config_path = 'experiment-configs/pliv-config.yaml'
results_dir = 'results/pliv'

def get_signed_bias(data, true, n):
    bias = np.mean((data[:, 0] - true))
    return f'{bias:.3f}'


def get_root_n_std(data, true, n):
    estimator_std = np.std(data[:, 0])
    return estimator_std * (n ** 0.5)


metric_funcs = {
    'bias': get_signed_bias,
    'root-n-std': get_root_n_std,
    # 'abs-bias': get_bias,
    # 'rmse': get_rmse,
}


with open(config_path) as f:
    config = yaml.safe_load(f)

setup_options = config['setup-options']

for clever in setup_options['clever']:
    print(f'clever = {clever}')
    resd = {}
    for dgp_version in setup_options['dgp-version']:
        print(f' - DGP version = {dgp_version}')
        resd[dgp_version] = {}
        for n in setup_options['n']:
            lambda_l2_h = .1/n**(.9)
            # print(lambda_l2_h)
            nkey = f'$n={n}$'
            resd[dgp_version][nkey] = {}
            assert len(setup_options['iv-strength'])== 1
            iv_strength = setup_options['iv-strength'][0]
            setup = {
                'clever': clever,
                'dgp-version': dgp_version,
                'n': n,
                'iv-strength': iv_strength,
            }
            results_fname = '__'.join(f'{k_}_{setup[k_]}' for k_ in setup_options.keys()) + '.jbl'
            results_path = os.path.join(results_dir, results_fname)
            true, results = joblib.load(results_path)
            resd[dgp_version][nkey] = {}
            for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
                # get array of results for given method over experiment replications
                data = np.array([r[it] for r in results])
                # print(f'result for method={method}, n={n}:')
                # print(f'true = {true}')
                # display(pd.Series(data[:, 0]).describe())
                resd[dgp_version][nkey][method] = {
                    metric_: metric_func_(data, true, n)
                    for metric_, metric_func_ in metric_funcs.items()
                }
                resd[dgp_version][nkey] = pd.DataFrame(resd[dgp_version][nkey])
        resd[dgp_version] = pd.concat(resd[dgp_version])

    display(pd.concat(resd).unstack(level=2))
    print(pd.concat(resd).unstack(level=2).to_latex(bold_rows=True, multirow=True,
                                                    multicolumn=True, escape=False,
                                                    column_format='ll||ll|ll|ll|ll|',
                                                    multicolumn_format='c|'))



clever = True
 - DGP version = DGP-1


dr                                 tmle                \
                  bias root-n-std abs-bias   rmse      bias    root-n-std   
DGP-1 $n=100$   -0.552   8.556239    0.552  1.018     0.325    265.280776   
      $n=500$   -0.192  19.777352    0.192  0.905     4.177   1698.433666   
      $n=1000$  -0.377  34.116817    0.377  1.143    -1.120     339.69932   
      $n=2000$  -0.390  64.818547    0.390  1.501  -186.636  84374.681066   

                                      ipw                               \
               abs-bias      rmse    bias  root-n-std abs-bias    rmse   
DGP-1 $n=100$     0.325    26.530  -0.919   20.987186    0.919   2.291   
      $n=500$     4.177    76.071  -1.911  158.834184    1.911   7.356   
      $n=1000$    1.120    10.800  -0.317  331.161631    0.317  10.477   
      $n=2000$  186.636  1895.884   0.364  755.656202    0.364  16.901   

                direct                             
                  bias root-n-std abs-bias   rmse  
DGP-1 $n=100$   -0.553   8.557271    0.553  1.019  
      $n=500$   -0.195  17.651085    0.195  0.813  
      $n=1000$  -0.278  25.654456    0.278  0.858  
      $n=2000$  -0.144  33.388127    0.144  0.760

\begin{tabular}{ll||llll|llll|llll|llll|}
\toprule
 &  & \multicolumn{4}{c|}{dr} & \multicolumn{4}{c|}{tmle} & \multicolumn{4}{c|}{ipw} & \multicolumn{4}{c|}{direct} \\
 &  & bias & root-n-std & abs-bias & rmse & bias & root-n-std & abs-bias & rmse & bias & root-n-std & abs-bias & rmse & bias & root-n-std & abs-bias & rmse \\
\midrule
\multirow[t]{4}{*}{\textbf{DGP-1}} & \textbf{$n=100$} & -0.552 & 8.556239 & 0.552 & 1.018 & 0.325 & 265.280776 & 0.325 & 26.530 & -0.919 & 20.987186 & 0.919 & 2.291 & -0.553 & 8.557271 & 0.553 & 1.019 \\
\textbf{} & \textbf{$n=500$} & -0.192 & 19.777352 & 0.192 & 0.905 & 4.177 & 1698.433666 & 4.177 & 76.071 & -1.911 & 158.834184 & 1.911 & 7.356 & -0.195 & 17.651085 & 0.195 & 0.813 \\
\textbf{} & \textbf{$n=1000$} & -0.377 & 34.116817 & 0.377 & 1.143 & -1.120 & 339.699320 & 1.120 & 10.800 & -0.317 & 331.161631 & 0.317 & 10.477 & -0.278 & 25.654456 & 0.278 & 0.858 \\
\textbf{} & \textbf{$n=2000$} & -0.390 & 64.818547 & 0.390 & 1.501 & -186.636 & 84374.68106

dr                                 tmle                \
                  bias root-n-std abs-bias   rmse      bias    root-n-std   
DGP-1 $n=100$   -0.563   8.658977    0.563  1.033     1.470    292.865895   
      $n=500$   -0.624  23.597716    0.624  1.226     2.224   1623.295634   
      $n=1000$  -0.572  35.804127    0.572  1.268    -1.335    356.547114   
      $n=2000$  -0.402  78.384729    0.402  1.798  -208.770  95582.631729   

                                      ipw                               \
               abs-bias      rmse    bias  root-n-std abs-bias    rmse   
DGP-1 $n=100$     1.470    29.323  -0.919   20.987186    0.919   2.291   
      $n=500$     2.224    72.630  -1.911  158.834184    1.911   7.356   
      $n=1000$    1.335    11.354  -0.317  331.161631    0.317  10.477   
      $n=2000$  208.770  2147.465   0.364  755.656202    0.364  16.901   

                direct                             
                  bias root-n-std abs-bias   rmse  
DGP-1 $n=100$   -0.563   8.578198    0.563  1.026  
      $n=500$   -0.616  21.917372    0.616  1.158  
      $n=1000$  -0.534  26.987407    0.534  1.007  
      $n=2000$  -0.165  45.283072    0.165  1.026

\begin{tabular}{ll||llll|llll|llll|llll|}
\toprule
 &  & \multicolumn{4}{c|}{dr} & \multicolumn{4}{c|}{tmle} & \multicolumn{4}{c|}{ipw} & \multicolumn{4}{c|}{direct} \\
 &  & bias & root-n-std & abs-bias & rmse & bias & root-n-std & abs-bias & rmse & bias & root-n-std & abs-bias & rmse & bias & root-n-std & abs-bias & rmse \\
\midrule
\multirow[t]{4}{*}{\textbf{DGP-1}} & \textbf{$n=100$} & -0.563 & 8.658977 & 0.563 & 1.033 & 1.470 & 292.865895 & 1.470 & 29.323 & -0.919 & 20.987186 & 0.919 & 2.291 & -0.563 & 8.578198 & 0.563 & 1.026 \\
\textbf{} & \textbf{$n=500$} & -0.624 & 23.597716 & 0.624 & 1.226 & 2.224 & 1623.295634 & 2.224 & 72.630 & -1.911 & 158.834184 & 1.911 & 7.356 & -0.616 & 21.917372 & 0.616 & 1.158 \\
\textbf{} & \textbf{$n=1000$} & -0.572 & 35.804127 & 0.572 & 1.268 & -1.335 & 356.547114 & 1.335 & 11.354 & -0.317 & 331.161631 & 0.317 & 10.477 & -0.534 & 26.987407 & 0.534 & 1.007 \\
\textbf{} & \textbf{$n=2000$} & -0.402 & 78.384729 & 0.402 & 1.798 & -208.770 & 95582.63172

In [16]:
# results for average derivative on DGP in CCT paper

config_path = 'experiment-configs/cct-config.yaml'
results_dir = 'results/cct'

def get_mean(data, true, n):
    return data[:, 0].mean()


def get_std(data, true, n):
    return data[:, 0].std()


metric_funcs = {
    'mean': get_mean,
    'std': get_std,
}


with open(config_path) as f:
    config = yaml.safe_load(f)

setup_options = config['setup-options']

for clever in setup_options['clever']:
    print(f'clever = {clever}')
    resd = {}
    for dgp_version in setup_options['dgp-version']:
        print(f' - DGP version = {dgp_version}')
        resd[dgp_version] = {}
        for n in setup_options['n']:
            lambda_l2_h = .1/n**(.9)
            # print(lambda_l2_h)
            nkey = f'$n={n}$'
            resd[dgp_version][nkey] = {}
            assert len(setup_options['iv-strength'])== 1
            iv_strength = setup_options['iv-strength'][0]
            setup = {
                'clever': clever,
                'dgp-version': dgp_version,
                'n': n,
                'iv-strength': iv_strength,
            }
            results_fname = '__'.join(f'{k_}_{setup[k_]}' for k_ in setup_options.keys()) + '.jbl'
            results_path = os.path.join(results_dir, results_fname)
            true, results = joblib.load(results_path)
            resd[dgp_version][nkey] = {}
            for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
                # get array of results for given method over experiment replications
                data = np.array([r[it] for r in results])
                # print(f'result for method={method}, n={n}:')
                # print(f'true = {true}')
                # display(pd.Series(data[:, 0]).describe())
                resd[dgp_version][nkey][method] = {
                    metric_: metric_func_(data, true, n)
                    for metric_, metric_func_ in metric_funcs.items()
                }
                resd[dgp_version][nkey] = pd.DataFrame(resd[dgp_version][nkey])
        resd[dgp_version] = pd.concat(resd[dgp_version])

    display(pd.concat(resd).unstack(level=2))
    print(pd.concat(resd).unstack(level=2).to_latex(bold_rows=True, multirow=True,
                                                    multicolumn=True, escape=False,
                                                    column_format='ll||ll|ll|ll|ll|',
                                                    multicolumn_format='c|'))



clever = True
 - DGP version = MC2


dr                tmle                 ipw            \
                  mean       std      mean       std      mean       std   
MC2 $n=1000$  1.150949  0.158529  1.173178  0.143926  1.546083  4.798609   
    $n=5000$  1.066365  0.107496  1.133543  0.078166  1.750267  5.529226   

                direct            
                  mean       std  
MC2 $n=1000$  1.294742  0.081570  
    $n=5000$  1.276180  0.043668

\begin{tabular}{ll||ll|ll|ll|ll|}
\toprule
 &  & \multicolumn{2}{c|}{dr} & \multicolumn{2}{c|}{tmle} & \multicolumn{2}{c|}{ipw} & \multicolumn{2}{c|}{direct} \\
 &  & mean & std & mean & std & mean & std & mean & std \\
\midrule
\multirow[t]{2}{*}{\textbf{MC2}} & \textbf{$n=1000$} & 1.150949 & 0.158529 & 1.173178 & 0.143926 & 1.546083 & 4.798609 & 1.294742 & 0.081570 \\
\textbf{} & \textbf{$n=5000$} & 1.066365 & 0.107496 & 1.133543 & 0.078166 & 1.750267 & 5.529226 & 1.276180 & 0.043668 \\
\cline{1-10}
\bottomrule
\end{tabular}

clever = False
 - DGP version = MC2


dr                tmle                 ipw            \
                  mean       std      mean       std      mean       std   
MC2 $n=1000$  1.161099  0.145584  1.157694  0.139463  1.546083  4.798609   
    $n=5000$  1.117925  0.109414  1.114984  0.077220  1.750267  5.529226   

                direct            
                  mean       std  
MC2 $n=1000$  1.179854  0.158979  
    $n=5000$  1.111690  0.102274

\begin{tabular}{ll||ll|ll|ll|ll|}
\toprule
 &  & \multicolumn{2}{c|}{dr} & \multicolumn{2}{c|}{tmle} & \multicolumn{2}{c|}{ipw} & \multicolumn{2}{c|}{direct} \\
 &  & mean & std & mean & std & mean & std & mean & std \\
\midrule
\multirow[t]{2}{*}{\textbf{MC2}} & \textbf{$n=1000$} & 1.161099 & 0.145584 & 1.157694 & 0.139463 & 1.546083 & 4.798609 & 1.179854 & 0.158979 \\
\textbf{} & \textbf{$n=5000$} & 1.117925 & 0.109414 & 1.114984 & 0.077220 & 1.750267 & 5.529226 & 1.111690 & 0.102274 \\
\cline{1-10}
\bottomrule
\end{tabular}

